# Exercise 1  

**USE THE DOUBLE DISPATCH APPROACH**

Use abstract classes if needed.

Consider the following objects and their operations
`MyList`, `MyNumber`

`MyList + MyList -> MyList` 

The new list is composed of the sum of the homologous elements if the lists have the same length, otherwise the returned object is a `MyList` with 0 elements.

`MyNumber + MyNumber -> MyNumber` 

The new number is the sum of the previous ones.

`MyList + MyNumber -> MyList`

`MyNumber  + MyList -> MyList`

The new list is composed of the sum of the elements of `MyList` with `MyNumber`

Implement `MyList` and `MyNumber` using Composition. 
Implement `__add__()` using Double Dispatch.

In [16]:
from abc import ABC, abstractmethod

class MyElement(ABC):
    @abstractmethod 
    def __add__(self, other_element):
        pass

    @abstractmethod
    def _add_to_my_list(self, other_element):
        pass
    
    @abstractmethod
    def _add_to_my_number(self, other_element):
        pass


class MyList(MyElement):

    def __init__(self, l: list) -> None:
        self._list = l

    def __repr__(self):
        return self._list.__repr__()

    def __len__(self):
        return len(self._list)

    def __add__(self, other_element: MyElement):
        return other_element._add_to_my_list(self)

    def _add_to_my_list(self, other_element):
        if len(other_element) == len(self):
            return MyList([i + j for i, j in zip(self._list, other_element._list)])
        else:
            return MyList([])

    def _add_to_my_number(self, other_element):
        return MyList([i + other_element._number for i in self._list])

class MyNumber(MyElement):

    def __init__(self, n: float):
        self._number = n
    
    def __repr__(self):
        return self._number.__repr__()

    def __add__(self, other_element: MyElement) -> MyElement:
        return other_element._add_to_my_number(self)

    def _add_to_my_list(self, other_element: MyList) -> MyList:
        return MyList([i + self._number for i in other_element._list])

    def _add_to_my_number(self, other_element):
        return MyNumber(self._number + other_element._number)



In [17]:
# Example
myList1 = MyList([1,2,3])
myList2 = MyList([4,5,6])
myList3 = MyList([4,5,6,7])
myNumber1 = MyNumber(10)
myNumber2 = MyNumber(20)

	
myList4 = myList1 + myList2 # note: you can use the magic method __add__()
print(myList4) # -> [5,7,9]


myList5 = myList1 + myList3 
print(myList5) # -> []

myNumber3 = myNumber1 + myNumber2

print(myNumber3) # 30

myList6 = myList1 + myNumber1
print(myList6) # -> [11, 12, 13]

myList7 = myNumber1 + myList1
print(myList7) # -> [11, 12, 13]


[5, 7, 9]
[]
30
[11, 12, 13]
[11, 12, 13]


# Exercise 2 

**USE THE STATE DESIGN PATTERN and the OBSERVER DESIGN PATTERN**

Use abstract classes if needed.


An object receives signals 0, 1, 2 using the `signal()` method, and characters (one character at a time) using the `c_input()` method.

In the initial state, the object prints each character received. If it receives signals `0, 1, 2` exactly in this order, it goes into a final state. 

In the final state the object prints the character following the input character. Assume the character after `'z'` is `'a'`.

`c_input(‘a’) -> 'b'`

`c_input(‘z’) -> 'a'`

The 0 signal induces a transition to the initial state. The other signals have no effect (when the state is the initial one).

In intermediate states the object does not print the received characters.
Signals that break the correct sequence (i.e., `0, 1, 1, ...`) induce a transition to the initial state.

NB it is required to implement the object defining the intermediate states, not using a counter to manage the transition.

1)	Implement the requested behavior using the state design pattern. 

2)	Define 3 observers `o1`, `o2`, `o3` that respond to the state transition.


Write a main that clearly shows the transitions of the stateful object using various signal sequences. The main must also show the behaviour of the observers by subscribing and unsubscribing them.

In [20]:
class SignalState(ABC):
    @abstractmethod
    def signal(self, s, receiver):
        pass

    @staticmethod
    @abstractmethod
    def c_input(character):
        pass

class InitialState(SignalState):
    def signal(self, s, receiver):
        if s == 0:
            receiver._change_state(ReceivedZeroState())
    
    @staticmethod
    def c_input(character):
        print(character)

class ReceivedZeroState(SignalState):
    def signal(self, s, receiver):
        if s == 1:
            receiver._change_state(ReceivedOneState())
        else:
            receiver._change_state(InitialState())
    
    @staticmethod
    def c_input(character):
        pass
        
class ReceivedOneState(SignalState):
    def signal(self, s, receiver):
        if s == 2:
            receiver._change_state(FinalState())
        else:
            receiver._change_state(InitialState())
    
    @staticmethod
    def c_input(character):
        pass
    
class FinalState(SignalState):
    def signal(self, s, receiver):
        receiver._change_state(InitialState())
    
    @staticmethod
    def c_input(character):
        if ord(character) >= ord('a') and ord(character) < ord('z'):
            c = chr(ord(character) + 1)
        elif ord(character) == ord('z'):
            c = 'a'
        else:
            raise ValueError("Cannot print this character.")
        print(c)

class Receiver:
    def __init__(self):
        self._state = InitialState()

    def _change_state(self, new_state):
        self._state = new_state

    def signal(self, s):
        self._state.signal(s, self)

    def c_input(self, character):
        self._state.c_input(character) 


r = Receiver()
r.c_input("a")  # initial state "a"
r.signal(0)  # go to received zero state
r.c_input("a")  # received zero state no output
r.signal(1)  # go to received one state
r.signal(2)  # go to received two state a.k.a. final state
r.c_input("a") # final state "b"
r.signal(2)  # back to the initial state
r.c_input("a") # initial state "a"

a
b
a


In [21]:
"""This part is the code from before but also implement the observer pattern."""

class SignalState(ABC):
    @abstractmethod
    def signal(self, s, receiver):
        pass

    @staticmethod
    @abstractmethod
    def c_input(character):
        pass

class InitialState(SignalState):
    def signal(self, s, receiver):
        if s == 0:
            receiver._change_state(ReceivedZeroState())
    
    @staticmethod
    def c_input(character):
        print(character)

class ReceivedZeroState(SignalState):
    def signal(self, s, receiver):
        if s == 1:
            receiver._change_state(ReceivedOneState())
        else:
            receiver._change_state(InitialState())
    
    @staticmethod
    def c_input(character):
        pass
        
class ReceivedOneState(SignalState):
    def signal(self, s, receiver):
        if s == 2:
            receiver._change_state(FinalState())
        else:
            receiver._change_state(InitialState())
    
    @staticmethod
    def c_input(character):
        pass
    
class FinalState(SignalState):
    def signal(self, s, receiver):
        receiver._change_state(InitialState())
    
    @staticmethod
    def c_input(character):
        if ord(character) >= ord('a') and ord(character) < ord('z'):
            c = chr(ord(character) + 1)
        elif ord(character) == ord('z'):
            c = 'a'
        else:
            raise ValueError("Cannot print this character.")
        print(c)

class Receiver:
    def __init__(self):
        self._state = InitialState()
        self.observers = set()

    def _change_state(self, new_state):
        self._state = new_state
        self.dispatch("new state: " + self._state.__class__.__name__)

    def signal(self, s):
        self._state.signal(s, self)

    def c_input(self, character):
        self._state.c_input(character) 

    def subscribe(self, observer):
        self.observers.add(observer)
    
    def unsubscribe(self, observer):
        if observer in self.observers:
            self.observers.remove(observer)
    
    def dispatch(self, message):
        for observer in self.observers:
            observer.update(message)



class ChangeStateObserver:
    def __init__(self, name):
        self.name = name

    def update(self, message):
        print(self.name + " " + message)


o1 = ChangeStateObserver("o1")
o2 = ChangeStateObserver("o2")
o3 = ChangeStateObserver("o3")

r = Receiver()
r.subscribe(o1)  # observers: o1
r.c_input("a")  # initial state "a"
r.signal(0)  # go to received zero state
r.c_input("a")  # received zero state no output
r.subscribe(o2)  # observers: o1, o2
r.signal(1)  # go to received one state
r.unsubscribe(o1)  # observers: o2
r.signal(2)  # go to received two state a.k.a. final state
r.c_input("a") # final state "b"
r.subscribe(o3)  # observers: o2, o3
r.signal(2)  # back to the initial state
r.c_input("a") # initial state "a"

a
o1 new state: ReceivedZeroState
o1 new state: ReceivedOneState
o2 new state: ReceivedOneState
o2 new state: FinalState
b
o3 new state: InitialState
o2 new state: InitialState
a
